In [12]:
import os
import numpy as np
import cv2
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from sklearn.model_selection import train_test_split

def load_images_from_directory(directory, target_size=(224, 224)):
    images = []
    labels = []
    class_labels = sorted(os.listdir(directory))  # Each person_name folder is a class label
    label_map = {label: idx for idx, label in enumerate(class_labels)}
    
    for person_name in class_labels:
        person_path = os.path.join(directory, person_name)
        if os.path.isdir(person_path):
            for hand in ['left', 'right']:
                hand_path = os.path.join(person_path, hand)
                if os.path.isdir(hand_path):
                    for finger in ['middle', 'ring', 'index']:
                        finger_path = os.path.join(hand_path, finger)
                        if os.path.isdir(finger_path):
                            for filename in os.listdir(finger_path):
                                if filename.endswith('.bmp'):
                                    img_path = os.path.join(finger_path, filename)
                                    try:
                                        # img = load_img(img_path, target_size=target_size)  # Load image
                                        img = cv2.imread(img_path) 
                                        img = cv2.resize(img, (224,224)) 
                                        # _,img = cv2.threshold(img, 127, 255, cv2.THRESH_BINARY)
                                        img_array = img_to_array(img) / 255.0  # Normalize image
                                        images.append(img_array)
                                        labels.append(label_map[person_name])
                                    except Exception as e:
                                        print(f"Error loading image {img_path}: {e}")
    
    return np.array(images), np.array(labels)

# Example usage
directory = r"E:\Machine Learning Project\Finger Vein Database"  # Update with the actual path
X, y = load_images_from_directory(directory)

In [13]:
# Split into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [14]:
print("Dataset loaded successfully!")
print("Train set size:", X_train.shape)
print("Test set size:", X_test.shape)

Dataset loaded successfully!
Train set size: (1425, 224, 224, 3)
Test set size: (357, 224, 224, 3)


In [15]:
from skimage.feature import hog, local_binary_pattern
from skimage.filters import gabor
import numpy as np
from skimage.color import rgb2gray  # To convert RGB to grayscale

def extract_features(images):
    features = []
    
    for img in images:
        # Convert RGB image to grayscale
        img_gray = rgb2gray(img)  # Converts to 2D grayscale image
        
        # Convert to float32 and normalize to [0, 1]
        img_gray = img_gray.astype(np.float32)  # No need to normalize again if using rgb2gray
        
        # Compute HOG features
        try:
            hog_features = hog(img_gray, pixels_per_cell=(8, 8), cells_per_block=(2, 2), feature_vector=True)
        except Exception as e:
            print(f"HOG feature extraction failed for image {img.shape}: {e}")
            hog_features = np.zeros(0)  # Placeholder for failed extraction
        
        # Compute LBP features
        lbp = local_binary_pattern(img_gray, P=8, R=1, method="uniform")
        lbp_hist, _ = np.histogram(lbp.ravel(), bins=np.arange(0, 11), density=True)  # Normalize histogram
        
        # Compute Gabor features
        try:
            gabor_real, gabor_imag = gabor(img_gray, frequency=0.6)
            gabor_features = [np.mean(gabor_real), np.var(gabor_real), np.mean(gabor_imag), np.var(gabor_imag)]
        except Exception as e:
            print(f"Gabor feature extraction failed for image {img.shape}: {e}")
            gabor_features = [0, 0, 0, 0]  # Placeholder for failed extraction
        
        # Concatenate all features
        feature_vector = np.hstack([hog_features, lbp_hist, gabor_features])
        features.append(feature_vector)
    
    return np.array(features)

# Extract features from X_train
X_train_features = extract_features(X_train)

print("Feature extraction completed!")
print("Feature shape:", X_train_features.shape)

Feature extraction completed!
Feature shape: (1425, 26258)


In [19]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical

# Define CNN Model
def build_model(input_shape, num_classes):
    model = Sequential()
    model.add(Dense(1024, input_dim=input_shape[0], activation='relu'))  # Increase the number of neurons
    model.add(Dropout(0.5))
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(num_classes, activation='softmax'))  # Output layer for classification
    
    model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# Prepare the labels (make sure they are one-hot encoded)
num_classes = 50   # Update this value to match the number of classes in your dataset
y_train = to_categorical(y_train, num_classes=num_classes)  # Assuming you have 'y_train' labels available

# Build the model
model = build_model(input_shape=(X_train_features.shape[1],), num_classes=num_classes)  # Update num_classes here

# Train the model
model.fit(X_train_features, y_train, epochs=100, batch_size=32, validation_split=0.2)

# Evaluate the model
loss, accuracy = model.evaluate(X_train_features, y_train)
print(f"Model Loss: {loss}, Model Accuracy: {accuracy}")

Epoch 1/100
36/36 ━━━━━━━━━━━━━━━━━━━━ 23s 556ms/step - accuracy: 0.0139 - loss: 5.8400 - val_accuracy: 0.0421 - val_loss: 3.8977
Epoch 2/100
36/36 ━━━━━━━━━━━━━━━━━━━━ 20s 547ms/step - accuracy: 0.0290 - loss: 4.0788 - val_accuracy: 0.0561 - val_loss: 3.8586
Epoch 3/100
36/36 ━━━━━━━━━━━━━━━━━━━━ 20s 546ms/step - accuracy: 0.0317 - loss: 3.9705 - val_accuracy: 0.0386 - val_loss: 3.8425
Epoch 4/100
36/36 ━━━━━━━━━━━━━━━━━━━━ 20s 547ms/step - accuracy: 0.0408 - loss: 3.9092 - val_accuracy: 0.0561 - val_loss: 3.7968
Epoch 5/100
36/36 ━━━━━━━━━━━━━━━━━━━━ 20s 550ms/step - accuracy: 0.0411 - loss: 3.8495 - val_accuracy: 0.0561 - val_loss: 3.6657
Epoch 6/100
36/36 ━━━━━━━━━━━━━━━━━━━━ 20s 560ms/step - accuracy: 0.0494 - loss: 3.7659 - val_accuracy: 0.0667 - val_loss: 3.6351
Epoch 7/100
36/36 ━━━━━━━━━━━━━━━━━━━━ 20s 545ms/step - accuracy: 0.0581 - loss: 3.6926 - val_accuracy: 0.0561 - val_loss: 3.5082
Epoch 8/100
36/36 ━━━━━━━━━━━━━━━━━━━━ 20s 547ms/step - accuracy: 0.0519 - loss: 3.5798 - 

In [22]:
model.save('model.h5')  # Saves model as a single file
